In [154]:
import pandas as pd

# Load the CSV file (ensure the file is in the same directory as your notebook)
df = pd.read_csv("Meteorite_Landings.csv")

# Display the first 5 records
print("First 5 records:")
print(df.head())


First 5 records:
       name   id nametype     recclass  mass (g)  fall    year    reclat  \
0    Aachen    1    Valid           L5      21.0  Fell  1880.0  50.77500   
1    Aarhus    2    Valid           H6     720.0  Fell  1951.0  56.18333   
2      Abee    6    Valid          EH4  107000.0  Fell  1952.0  54.21667   
3  Acapulco   10    Valid  Acapulcoite    1914.0  Fell  1976.0  16.88333   
4   Achiras  370    Valid           L6     780.0  Fell  1902.0 -33.16667   

     reclong           GeoLocation  Unnamed: 10  
0    6.08333     (50.775, 6.08333)          NaN  
1   10.23333  (56.18333, 10.23333)          NaN  
2 -113.00000    (54.21667, -113.0)          NaN  
3  -99.90000     (16.88333, -99.9)          NaN  
4  -64.95000   (-33.16667, -64.95)          NaN  


In [155]:
# Check for duplicates based on the 'id' column
duplicate_count = df.duplicated(subset=["id"]).sum()
print(f"\nNumber of duplicate records based on 'id': {duplicate_count}")

# Drop duplicates if necessary
df = df.drop_duplicates(subset=["id"])

# Check for missing values in each column
print("\nMissing values per column:")
print(df.isnull().sum())


Number of duplicate records based on 'id': 0

Missing values per column:
name               0
id                 0
nametype           0
recclass           0
mass (g)         131
fall               0
year             291
reclat          7315
reclong         7315
GeoLocation     7315
Unnamed: 10    45716
dtype: int64


In [156]:
df.drop(columns=["Unnamed: 10", "unknown"], inplace=True, errors='ignore')

# 2. Drop all rows that have *any* missing values in *any* column
df.dropna(how='any', inplace=True)

# 3. Check how many rows remain
print(f"Number of rows left after dropping rows with missing values: {len(df)}")

# Optional: Display the updated DataFrame structure
print("\nUpdated DataFrame info:")
print(df.info())
print("\nFirst 5 rows of the updated DataFrame:")
print(df.head())

Number of rows left after dropping rows with missing values: 38115

Updated DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 38115 entries, 0 to 45715
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         38115 non-null  object 
 1   id           38115 non-null  int64  
 2   nametype     38115 non-null  object 
 3   recclass     38115 non-null  object 
 4   mass (g)     38115 non-null  float64
 5   fall         38115 non-null  object 
 6   year         38115 non-null  float64
 7   reclat       38115 non-null  float64
 8   reclong      38115 non-null  float64
 9   GeoLocation  38115 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 3.2+ MB
None

First 5 rows of the updated DataFrame:
       name   id nametype     recclass  mass (g)  fall    year    reclat  \
0    Aachen    1    Valid           L5      21.0  Fell  1880.0  50.77500   
1    Aarhus    2    Valid           H6   

In [157]:

# We assume df is already loaded and missing values dropped
print("Data Types Before:")
print(df.dtypes)

# 1. Convert 'id' to int if it’s truly an integer ID
df["id"] = df["id"].astype(int)

# 2. Convert 'year' to integer if it’s just a year (e.g., 1990)
#    or to datetime if it’s a full date. Let's assume it's just a year:
df["year"] = df["year"].astype(int)

print("\nData Types After:")
print(df.dtypes)


Data Types Before:
name            object
id               int64
nametype        object
recclass        object
mass (g)       float64
fall            object
year           float64
reclat         float64
reclong        float64
GeoLocation     object
dtype: object

Data Types After:
name            object
id               int64
nametype        object
recclass        object
mass (g)       float64
fall            object
year             int64
reclat         float64
reclong        float64
GeoLocation     object
dtype: object


In [158]:
invalid_mass = df[df["mass (g)"] < 0]
if not invalid_mass.empty:
    print("\nRows with negative mass:")
    print(invalid_mass)
    # Optionally drop or fix these rows
    # df = df[df["mass (g)"] >= 0]


In [159]:
invalid_mass = df[df["mass (g)"] < 0]
if not invalid_mass.empty:
    print("\nRows with negative mass:")
    print(invalid_mass)
    # Optionally drop or fix these rows
    # df = df[df["mass (g)"] >= 0]


In [160]:
invalid_lat = df[(df["reclat"] < -90) | (df["reclat"] > 90)]
invalid_lon = df[(df["reclong"] < -180) | (df["reclong"] > 180)]

if not invalid_lat.empty or not invalid_lon.empty:
    print("\nInvalid lat/long rows:")
    print(pd.concat([invalid_lat, invalid_lon]).drop_duplicates())
    # Optionally drop them or fix them:
    # df = df[(df["reclat"] >= -90) & (df["reclat"] <= 90) &
    #         (df["reclong"] >= -180) & (df["reclong"] <= 180)]


In [161]:
# Example: Keep only years between 800 and 2025
invalid_years = df[(df["year"] < 800) | (df["year"] > 2025)]
if not invalid_years.empty:
    print("\nRows with invalid years:")
    print(invalid_years)
    # Optionally drop them:
    # df = df[(df["year"] >= 800) & (df["year"] <= 2025)]



Rows with invalid years:
                        name     id nametype recclass  mass (g)   fall  year  \
30682  Northwest Africa 7701  57150    Valid      CK6      55.0  Found  2101   

       reclat  reclong GeoLocation  
30682     0.0      0.0  (0.0, 0.0)  


In [162]:
print("\nUnique values in 'fall':")
print(df["fall"].unique())

# If you see weird typos or extra spaces, you can standardize:
df["fall"] = df["fall"].str.strip().str.title()  # e.g. "Fell", "Found"
valid_fall_values = ["Fell", "Found"]

invalid_fall = df[~df["fall"].isin(valid_fall_values)]
if not invalid_fall.empty:
    print("\nInvalid 'fall' values found:")
    print(invalid_fall["fall"].value_counts())
    # Optionally drop or correct them
    # df = df[df["fall"].isin(valid_fall_values)]



Unique values in 'fall':
['Fell' 'Found']


In [163]:
df["recclass"] = df["recclass"].str.strip()

print("\nSome unique recclass values:")
print(df["recclass"].unique()[:20])  # Show first 20 unique classes



Some unique recclass values:
['L5' 'H6' 'EH4' 'Acapulcoite' 'L6' 'LL3-6' 'H5' 'L' 'Diogenite-pm' 'H4'
 'H' 'Iron, IVA' 'CR2-an' 'LL5' 'CI1' 'L/LL4' 'Eucrite-mmict' 'CV3'
 'Ureilite-an' 'Stone-uncl']


In [164]:
# Drop all rows outside the 800–2025 range
df = df[(df["year"] >= 800) & (df["year"] <= 2025)]

print("Rows remaining after dropping invalid years:", len(df))


Rows remaining after dropping invalid years: 38114


In [165]:
# Clip the 'year' values between 800 and 2025
df["year"] = df["year"].clip(lower=800, upper=2025)

# Verify if clipping worked
invalid_years = df[(df["year"] < 800) | (df["year"] > 2025)]
print("Number of invalid years after clipping:", len(invalid_years))


Number of invalid years after clipping: 0


In [166]:
# Check the first few values of GeoLocation
print(df["GeoLocation"].head(10))

# Check how many unique values
print("\nNumber of unique GeoLocation values:", df["GeoLocation"].nunique())

# Check for nulls
print("\nNumber of nulls in GeoLocation:", df["GeoLocation"].isnull().sum())


0       (50.775, 6.08333)
1    (56.18333, 10.23333)
2      (54.21667, -113.0)
3       (16.88333, -99.9)
4     (-33.16667, -64.95)
5            (32.1, 71.8)
6    (44.83333, 95.16667)
7     (44.21667, 0.61667)
8      (-31.6, -65.23333)
9     (-30.86667, -64.55)
Name: GeoLocation, dtype: object

Number of unique GeoLocation values: 16907

Number of nulls in GeoLocation: 0


In [167]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38114 entries, 0 to 45715
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         38114 non-null  object 
 1   id           38114 non-null  int64  
 2   nametype     38114 non-null  object 
 3   recclass     38114 non-null  object 
 4   mass (g)     38114 non-null  float64
 5   fall         38114 non-null  object 
 6   year         38114 non-null  int64  
 7   reclat       38114 non-null  float64
 8   reclong      38114 non-null  float64
 9   GeoLocation  38114 non-null  object 
dtypes: float64(3), int64(2), object(5)
memory usage: 3.2+ MB


In [168]:
print(len(df))

38114


In [169]:
num_rows = len(df)
num_unique_ids = df["id"].nunique()

if num_unique_ids == num_rows:
    print("Yes, 'id' is unique for every row in the dataset.")
else:
    print(f"No, 'id' is not unique. Unique IDs: {num_unique_ids}, Total Rows: {num_rows}")


Yes, 'id' is unique for every row in the dataset.


In [170]:
# Sort the DataFrame by 'id' in increasing order
df.sort_values(by='id', inplace=True)

# Display the first 5 rows to verify
print("First 5 rows after sorting by 'id':")
print(df.head())


First 5 rows after sorting by 'id':
        name  id nametype recclass  mass (g)   fall  year    reclat  \
0     Aachen   1    Valid       L5      21.0   Fell  1880  50.77500   
1     Aarhus   2    Valid       H6     720.0   Fell  1951  56.18333   
1111   Abajo   4    Valid       H5     331.0  Found  1982  26.80000   
1113  Abbott   5    Valid     H3-6   21100.0  Found  1951  36.30000   
2       Abee   6    Valid      EH4  107000.0   Fell  1952  54.21667   

        reclong           GeoLocation  
0       6.08333     (50.775, 6.08333)  
1      10.23333  (56.18333, 10.23333)  
1111 -105.41667    (26.8, -105.41667)  
1113 -104.28333    (36.3, -104.28333)  
2    -113.00000    (54.21667, -113.0)  


In [171]:
# Assuming your DataFrame is called df
df.rename(columns={
    "id": "meteorite_id",
    "name": "meteorite_name",
    "recclass": "class",
    "mass (g)": "mass",
    "fall": "fall_type",
    "year": "fall_year",
    "reclat": "latitude",
    "reclong": "longitude"
}, inplace=True)

# Verify the new column names
print("Columns after renaming:")
print(df.columns)


Columns after renaming:
Index(['meteorite_name', 'meteorite_id', 'nametype', 'class', 'mass',
       'fall_type', 'fall_year', 'latitude', 'longitude', 'GeoLocation'],
      dtype='object')


In [172]:
df["class_id"] = df["meteorite_id"]
df["location_id"] = df["meteorite_id"]

# Verify the new columns
print(df.head())


     meteorite_name  meteorite_id nametype class      mass fall_type  \
0            Aachen             1    Valid    L5      21.0      Fell   
1            Aarhus             2    Valid    H6     720.0      Fell   
1111          Abajo             4    Valid    H5     331.0     Found   
1113         Abbott             5    Valid  H3-6   21100.0     Found   
2              Abee             6    Valid   EH4  107000.0      Fell   

      fall_year  latitude  longitude           GeoLocation  class_id  \
0          1880  50.77500    6.08333     (50.775, 6.08333)         1   
1          1951  56.18333   10.23333  (56.18333, 10.23333)         2   
1111       1982  26.80000 -105.41667    (26.8, -105.41667)         4   
1113       1951  36.30000 -104.28333    (36.3, -104.28333)         5   
2          1952  54.21667 -113.00000    (54.21667, -113.0)         6   

      location_id  
0               1  
1               2  
1111            4  
1113            5  
2               6  


In [173]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38114 entries, 0 to 30784
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   meteorite_name  38114 non-null  object 
 1   meteorite_id    38114 non-null  int64  
 2   nametype        38114 non-null  object 
 3   class           38114 non-null  object 
 4   mass            38114 non-null  float64
 5   fall_type       38114 non-null  object 
 6   fall_year       38114 non-null  int64  
 7   latitude        38114 non-null  float64
 8   longitude       38114 non-null  float64
 9   GeoLocation     38114 non-null  object 
 10  class_id        38114 non-null  int64  
 11  location_id     38114 non-null  int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 3.8+ MB


In [174]:
import pandas as pd

# 1. Read only the id + raw lat/lon from the original CSV
raw_loc = pd.read_csv(
    "Meteorite_Landings.csv",
    usecols=["id", "reclat", "reclong"]
).rename(columns={
    "id": "meteorite_id",
    "reclat": "latitude",
    "reclong": "longitude"
})

# 2. Your region/country helper
def approximate_region_country(lat, lon):
    if 24.5 <= lat <= 49.0 and -125.0 <= lon <= -66.9:
        return "North America", "USA"
    if -33.0 <= lat <= 5.0 and -74.0 <= lon <= -34.0:
        return "South America", "Brazil"
    if 35.0 <= lat <= 71.0 and -10.0 <= lon <= 40.0:
        return "Europe", "Germany"
    if -44.0 <= lat <= -10.0 and 112.0 <= lon <= 154.0:
        return "Oceania", "Australia"
    if 20.0 <= lat <= 55.0 and 60.0 <= lon <= 100.0:
        return "Asia", "China"
    if -35.0 <= lat <= 37.0 and -17.0 <= lon <= 51.0:
        return "Africa", "Egypt"
    if lat > 70:
        return "Arctic", "Arctic Region"
    if lat < -60:
        return "Antarctica", "Antarctica"
    if 10.0 <= lat <= 20.0 and -85.0 <= lon <= -60.0:
        return "Caribbean", "Caribbean Sea"
    if -10.0 <= lat <= 10.0 and 95.0 <= lon <= 141.0:
        return "Southeast Asia", "Indonesia"
    return "Other", "Unknown"

# 3. Build locations_data by iterating over raw_loc
locations_data = []
for row in raw_loc.itertuples(index=False):
    lat, lon = row.latitude, row.longitude
    if pd.isnull(lat) or pd.isnull(lon):
        region, country = "Other", "Unknown"
    else:
        region, country = approximate_region_country(lat, lon)
    locations_data.append({
        "location_id": row.meteorite_id,
        "latitude":    lat,
        "longitude":   lon,
        "region":      region,
        "country":     country
    })

# 4. Convert and write
locations_df = pd.DataFrame(locations_data)
print("Locations DataFrame shape:", locations_df.shape)
print(locations_df.head(10))
locations_df.to_csv("locations.csv", index=False)
print("Locations table created successfully!")


Locations DataFrame shape: (45716, 5)
   location_id  latitude  longitude         region  country
0            1  50.77500    6.08333         Europe  Germany
1            2  56.18333   10.23333         Europe  Germany
2            6  54.21667 -113.00000          Other  Unknown
3           10  16.88333  -99.90000          Other  Unknown
4          370 -33.16667  -64.95000          Other  Unknown
5          379  32.10000   71.80000           Asia    China
6          390  44.83333   95.16667           Asia    China
7          392  44.21667    0.61667         Europe  Germany
8          398 -31.60000  -65.23333  South America   Brazil
9          417 -30.86667  -64.55000  South America   Brazil
Locations table created successfully!


In [175]:
# Save the cleaned DataFrame to a CSV file


# ─── 1) BUILD CLASSIFICATION BEFORE dropping `class` ───
classification_df = (
    df[["class_id","class"]]
      .drop_duplicates()
      .rename(columns={"class":"class_name"}))
classification_df.to_csv("meteorite_classification.csv", index=False)

df.drop(columns=["class","latitude","longitude"], inplace=True)

df.to_csv("cleaned_meteorite_landings.csv", index=False)

print("Cleaned dataset saved successfully!")


Cleaned dataset saved successfully!


In [176]:
import pandas as pd
import numpy as np
import random
from datetime import datetime, timedelta
random.seed(42)


In [177]:
# Expanded list of countries
countries = [
    "USA", "UK", "Germany", "France", "Canada", "Australia", "Japan", "China", "India", "Brazil",
    "Spain", "Italy", "Sweden", "Norway", "Denmark", "Finland", "Netherlands", "Belgium", "Switzerland", "Austria"
]

# Possible prefixes for institution names
institution_prefixes = [
    "University of", "National Institute of", "Global Research Center of",
    "College of", "Academy of", "Royal Institute of", "International Lab for",
    "Center for Advanced"
]

# Possible fields for institution names
institution_fields = [
    "Astronomy", "Geoscience", "Meteorite Studies", "Planetary Science",
    "Cosmic Research", "Space Exploration", "Astrophysics", "Geology", "Geochemistry"
]

# Number of institutions you want
num_institutions = 38114

institutions_data = []
for i in range(1, num_institutions + 1):
    # Randomly build an institution name by combining prefix + field
    prefix = random.choice(institution_prefixes)
    field = random.choice(institution_fields)
    institution_name = f"{prefix} {field}"
    
    # Randomly pick a country
    institution_country = random.choice(countries)
    
    institutions_data.append({
        "institution_id": i,
        "institution_name": institution_name,
        "institution_country": institution_country
    })

institutions_df = pd.DataFrame(institutions_data)

print("Institutions DataFrame shape:", institutions_df.shape)
print(institutions_df.head(10))  # Show the first 10 rows for variety

# Optionally, save to CSV
institutions_df.to_csv("institutions.csv", index=False)


Institutions DataFrame shape: (38114, 3)
   institution_id                             institution_name  \
0               1              National Institute of Astronomy   
1               2                 College of Planetary Science   
2               3           National Institute of Geochemistry   
3               4              International Lab for Astronomy   
4               5      National Institute of Planetary Science   
5               6                   University of Geochemistry   
6               7      International Lab for Planetary Science   
7               8                         Academy of Astronomy   
8               9      International Lab for Space Exploration   
9              10  Global Research Center of Planetary Science   

  institution_country  
0               India  
1              Canada  
2             Germany  
3                 USA  
4               China  
5               Japan  
6             Denmark  
7           Australia  
8               

In [178]:
# We'll assume you already have institutions_df with:
# columns = ["institution_id", "institution_name", "institution_country"]
# and institution_id goes from 1 to 3000.

num_scientists = 38114

# Example lists of first and last names
# A list of 100 diverse first names
first_names = [
    "Alice", "Bob", "Carol", "David", "Eva", "Frank", "Grace", "Henry", "Irene", "Jack",
    "Kathy", "Larry", "Megan", "Nathan", "Olivia", "Paul", "Quincy", "Rachel", "Sam", "Tina",
    "Ursula", "Victor", "Wendy", "Xander", "Yvonne", "Zack", "Amira", "Bao", "Carmen", "Darius",
    "Ethan", "Fatima", "George", "Hannah", "Isabella", "Jamal", "Kofi", "Laila", "Mohammed", "Naomi",
    "Omar", "Priya", "Quinn", "Ravi", "Sofia", "Tariq", "Uma", "Viktor", "Xia", "Yara",
    "Zainab", "Abdul", "Bella", "Chloe", "Diego", "Elena", "Felix", "Gabriel", "Harper", "Ibrahim",
    "Jasmine", "Keiko", "Luis", "Mariana", "Nina", "Oscar", "Penelope", "Rashid", "Sara", "Tomas",
    "Umar", "Valentina", "Willow", "Ximena", "Yusuf", "Zara", "Aisha", "Brandon", "Ciara", "Dmitri",
    "Elijah", "Fiona", "Giovanni", "Helena", "Ivy", "Javier", "Kaori", "Leonardo", "Maya", "Nadia",
    "Orlando", "Priyanka", "Riley", "Santiago", "Tatiana", "Uriel", "Veronica", "Xavier", "Yasmin", "Zoe"
]

# A list of 100 diverse last names
last_names = [
    "Smith", "Johnson", "Williams", "Brown", "Jones", "Miller", "Davis", "Garcia", "Rodriguez", "Wilson",
    "Martinez", "Anderson", "Taylor", "Thomas", "Hernandez", "Moore", "Martin", "Jackson", "Thompson", "White",
    "Lopez", "Lee", "Gonzalez", "Harris", "Clark", "Lewis", "Robinson", "Walker", "Perez", "Hall",
    "Wright", "King", "Scott", "Green", "Baker", "Adams", "Nelson", "Hill", "Ramirez", "Campbell",
    "Mitchell", "Roberts", "Carter", "Phillips", "Evans", "Turner", "Torres", "Parker", "Collins", "Edwards",
    "Stewart", "Flores", "Morris", "Nguyen", "Murphy", "Rivera", "Cook", "Rogers", "Morgan", "Peterson",
    "Cooper", "Reed", "Bailey", "Bell", "Gomez", "Kelly", "Howard", "Ward", "Cox", "Diaz",
    "Richardson", "Wood", "Watson", "Brooks", "Bennett", "Gray", "James", "Reyes", "Cruz", "Hughes",
    "Price", "Myers", "Long", "Foster", "Sanders", "Ross", "Morales", "Powell", "Sullivan", "Russell",
    "Ortiz", "Jenkins", "Gutierrez", "Perry", "Butler", "Barnes", "Fisher", "Henderson", "Coleman", "Simmons"
]

scientists_data = []

# If you want to ensure you reference the actual IDs in institutions_df:
institution_ids = institutions_df["institution_id"].tolist()

for i in range(1, num_scientists + 1):
    first = random.choice(first_names)
    last = random.choice(last_names)
    name = f"{first} {last}"
    
    # Randomly pick an institution_id from the Institutions table
    inst_id = random.choice(institution_ids)
    
    # Generate a synthetic email
    email = f"{first.lower()}.{last.lower()}{i}@gmail.com"
    
    scientists_data.append({
        "scientist_id": i,
        "scientist_name": name,
        "institution_id": inst_id,
        "email": email
    })

scientists_df = pd.DataFrame(scientists_data)

print("Scientists DataFrame shape:", scientists_df.shape)
print(scientists_df.head(10))

# Optional: save to CSV
scientists_df.to_csv("scientists.csv", index=False)
print("\nScientists table created successfully!")


Scientists DataFrame shape: (38114, 4)
   scientist_id     scientist_name  institution_id  \
0             1        Naomi Gomez           14706   
1             2        Eva Morales            5977   
2             3      Zack Thompson            5123   
3             4  Isabella Thompson           12391   
4             5        Ethan Green           22064   
5             6        Javier King           31748   
6             7        Uma Johnson            9801   
7             8    Priyanka Taylor              54   
8             9     Ethan Thompson           17380   
9            10      George Taylor           15543   

                          email  
0        naomi.gomez1@gmail.com  
1        eva.morales2@gmail.com  
2      zack.thompson3@gmail.com  
3  isabella.thompson4@gmail.com  
4        ethan.green5@gmail.com  
5        javier.king6@gmail.com  
6        uma.johnson7@gmail.com  
7    priyanka.taylor8@gmail.com  
8     ethan.thompson9@gmail.com  
9     george.taylor10@gmai

In [179]:
import pandas as pd
import random

# 1. Build a quick lookup: meteorite_id -> fall_year from df
year_lookup = df.set_index("meteorite_id")["fall_year"].to_dict()

# 2. Set the number of finding records equal to the number of rows in df
num_findings = len(df)  # ensures one finding per meteorite row

# 3. Possible "findings" text
findings_options = [
    "Nickel traces found",
    "Unusual chemical makeup",
    "High metal content",
    "Low density",
    "Carbonaceous composition",
    "Silicate rich",
    "High iron content",
    "Magnetic properties observed",
    "Rare isotopes detected",
    "Significant sulfur levels",
    "Anomalous oxygen isotope ratio",
    "Presence of organic compounds",
    "High chromium content",
    "Elevated manganese levels",
    "Trace platinum group metals",
    "Low volatile compounds",
    "Unique mineral structure",
    "Crystalline texture observed",
    "Shock-metamorphosed structure",
    "Presence of water-bearing minerals",
    "Exotic mineral inclusions",
    "Abundant olivine crystals",
    "Anomalous silicon content",
    "High magnesium content",
    "Significant phosphorous levels",
    "Evidence of partial melting",
    "Porous texture noted",
    "High titanium concentration",
    "Iron-nickel alloy composition",
    "Low carbon content",
    "Elevated cobalt levels",
    "Trace rare earth elements"
]


# 4. Collect the list of valid meteorite IDs from df
meteorite_ids = df["meteorite_id"].tolist()

# 5. Generate the Findings data so that every meteorite has one record
findings_data = []
for i, m_id in enumerate(meteorite_ids, start=1):
    # Use the corresponding fall_year for this meteorite_id
    discovery_year = year_lookup[m_id]
    
    findings_data.append({
        "finding_id": i,
        "meteorite_id": m_id,
        "findings": random.choice(findings_options),
        "discovery_year": discovery_year
    })

# 6. Create the Findings DataFrame
findings_df = pd.DataFrame(findings_data)

# 7. Inspect the Findings DataFrame
print("Findings DataFrame shape:", findings_df.shape)
print(findings_df.head(10))
print("\nFindings table created successfully with no NaN values!")
findings_df.to_csv("findings.csv", index=False)

Findings DataFrame shape: (38114, 4)
   finding_id  meteorite_id                            findings  \
0           1             1                       Silicate rich   
1           2             2              Low volatile compounds   
2           3             4           Trace rare earth elements   
3           4             5  Presence of water-bearing minerals   
4           5             6       Iron-nickel alloy composition   
5           6             7               High chromium content   
6           7             8            Unique mineral structure   
7           8             9       Shock-metamorphosed structure   
8           9            10       Iron-nickel alloy composition   
9          10            11                Porous texture noted   

   discovery_year  
0            1880  
1            1951  
2            1982  
3            1951  
4            1952  
5            1941  
6            1840  
7            1997  
8            1976  
9            1989  

Find

In [180]:
# Number of funding records you want
num_funding = 38114

# Possible sponsors
sponsors = [
    "NASA", "NSF", "ESA", "Private", "University",
    "SpaceX", "Blue Origin", "Roscosmos", "CNSA", "ISRO",
    "JAXA", "CNES", "ASI", "KARI", "Sierra Nevada Corporation",
    "Lockheed Martin", "Boeing", "Airbus", "Northrop Grumman", "Raytheon",
    "General Dynamics", "Virgin Galactic", "SpaceIL", "Orbital ATK", "Thales Alenia Space",
    "Mitsubishi Heavy Industries", "Arianespace", "Dynetics", "Relativity Space", "Rocket Lab",
    "Firefly Aerospace", "PLD Space", "ExPace", "Effective Innovations", "Xplore",
    "Axiom Space", "MDA", "Maxar Technologies", "SES", "Intelsat",
    "Inmarsat", "Eutelsat", "Telesat", "OneWeb", "Hughes",
    "Globalstar", "Iridium", "Viasat", "Planet Labs", "Spire Global",
    "Astrocast", "Fleet Space Technologies", "GomSpace", "Kepler Communications", "Swarm Technologies",
    "Vector Launch", "Orbit Fab", "Made In Space", "Momentus", "SpacePharma",
    "Zero 2 Infinity", "Blue Canyon Technologies", "Tyvak Nano-Satellite Systems", "Aerospace Corporation", "Boeing Defense",
    "Lockheed Martin Space", "Northrop Grumman Innovation Systems", "Space Systems Loral", "SSL", "Paragon Space Development Corporation",
    "Bigelow Aerospace", "Aerojet Rocketdyne", "AeroVironment", "Ball Aerospace", "Cobham",
    "Cubic Corporation", "Diehl Aerospace", "Harris Corporation", "ITAR Solutions", "L3Harris",
    "OHB System", "Orbcomm", "Planetary Resources", "Rocket Crafters", "Space Adventures",
    "Space Angels", "Space Foundation", "Space Generation Advisory Council", "Society of Exploration Geophysicists", "The Planetary Society",
    "United Launch Alliance", "Vulcan Inc.", "Zero Gravity Solutions", "Exos Aerospace", "Astrobotic",
    "Firefly Rocket Technologies", "Relativity", "Interstellar", "NewSpace Capital", "Quantum Space"
]


# Collect a list of valid scientist IDs from scientists_df
scientist_ids = scientists_df["scientist_id"].tolist()

funding_data = []
for i in range(1, num_funding + 1):
    funding_data.append({
        "funding_id": i,
        "scientist_id": random.choice(scientist_ids),
        "amount": random.randint(100000, 500000),
        "sponsor": random.choice(sponsors),
        "year": random.randint(1900, 2025)
    })

funding_df = pd.DataFrame(funding_data)

print("Funding DataFrame shape:", funding_df.shape)
print(funding_df.head(10))

# Optionally, save to CSV
funding_df.to_csv("funding.csv", index=False)
print("\nFunding table created successfully!")


Funding DataFrame shape: (38114, 5)
   funding_id  scientist_id  amount         sponsor  year
0           1          3531  175476            CNSA  1960
1           2         37053  305660        Raytheon  1967
2           3          8877  377434        Momentus  2024
3           4            65  153250     Axiom Space  1996
4           5         33223  107847        Eutelsat  1930
5           6         23511  310861             MDA  1990
6           7         19703  284762    Interstellar  1997
7           8          2684  247397  Exos Aerospace  1941
8           9         36064  148904        L3Harris  1936
9          10         38114  430733         Orbcomm  1903

Funding table created successfully!


In [181]:
# We assume you have these DataFrames already in memory:
# findings_df (columns: finding_id, meteorite_id, findings, discovery_year)
# scientists_df (columns: scientist_id, scientist_name, institution_id, email)

# 1. Convert findings_df to a list of dictionaries for easy random sampling
findings_rows = findings_df.to_dict("records")

# 2. Collect all scientist IDs for random assignment
scientist_ids = scientists_df["scientist_id"].tolist()

# 3. Decide how many "study" records you want
num_studies = 38114

# 4. Date range for random study_date
start_date = datetime(2000, 1, 1)
end_date = datetime(2025, 12, 31)

def random_date(start, end):
    """Generate a random date between 'start' and 'end'."""
    delta = end - start
    random_days = random.randrange(delta.days)
    return start + timedelta(days=random_days)

studies_data = []

for i in range(1, num_studies + 1):
    # Pick a random row from the existing Findings table
    finding_row = random.choice(findings_rows)
    # This ensures the study references the same meteorite and findings text

    # Pick a random scientist
    s_id = random.choice(scientist_ids)

    # Generate a random date
    study_dt = random_date(start_date, end_date).strftime("%Y-%m-%d")

    # Build the Studies record
    studies_data.append({
        "study_id": i,
        "meteorite_id": finding_row["meteorite_id"],  # same meteorite as the chosen finding
        "scientist_id": s_id,
        "study_date": study_dt,
        "findings": finding_row["findings"]  # reuse the same text from Findings
    })

# 5. Convert to DataFrame
studies_df = pd.DataFrame(studies_data)
studies_df.drop(columns=["findings"], inplace=True)
# 6. Inspect
print("Studies DataFrame shape:", studies_df.shape)
print(studies_df.head(10))
print("\nStudies table created successfully!")

# 7. (Optional) save to CSV
studies_df.to_csv("studies.csv", index=False)


Studies DataFrame shape: (38114, 4)
   study_id  meteorite_id  scientist_id  study_date
0         1         14290         15519  2025-10-08
1         2         53592         33660  2019-06-14
2         3         24241         34891  2012-08-25
3         4          3580         36047  2018-02-07
4         5         20265          4053  2005-10-10
5         6          3597         30076  2012-09-28
6         7         16857          6119  2011-04-05
7         8          8042         27624  2015-08-28
8         9         46695          8126  2016-05-15
9        10         18189          9384  2008-02-15

Studies table created successfully!


In [182]:
# We'll assume you already have a DataFrame called scientists_df 
# with at least one column "scientist_id".

# Number of research papers you want
num_papers = 38114

# Example list of journal names
journals = [
    "Astrophysical Journal",
    "Nature",
    "Science",
    "Journal of Meteorites",
    "Geochimica et Cosmochimica Acta",
    "International Journal of Astronomy",
    "Space Science Reviews",
    "Journal of Planetary Science",
    "Astrophysics and Space Science",
    "Cosmic Research Journal",
    "Journal of Astrophysics",
    "Advances in Astronomy",
    "Journal of Space Exploration",
    "Journal of Cosmology",
    "Cosmochemistry and Planetary Materials",
    "Planetary and Space Science",
    "Astrochemistry Journal",
    "Space and Time Journal",
    "Journal of Exoplanet Studies",
    "Journal of Cosmic Evolution",
    "International Journal of Astrophysics",
    "Astrobiology Reviews",
    "Journal of Celestial Mechanics",
    "Space Exploration Today",
    "Advances in Space Research",
    "Journal of Cosmic Rays",
    "Galactic Science Journal",
    "Interstellar Research Letters",
    "Journal of Planetary Research",
    "Solar Physics Journal",
    "Journal of Astronomical Instrumentation",
    "Stellar Astronomy Journal",
    "Journal of Cosmological Studies",
    "Journal of Space and Astronomy",
    "Interplanetary Science",
    "Astronomy and Astrophysics",
    "Space Weather and Climate",
    "Journal of Solar and Stellar Studies",
    "Journal of Astrophysical Techniques",
    "Journal of Astronomical Observations",
    "Cosmic Horizons Journal",
    "Journal of Extragalactic Astronomy",
    "Deep Space Journal",
    "Journal of Astrostatistics",
    "Space Science and Technology",
    "Astrophysical Research Letters",
    "Journal of Astronomical Sciences",
    "Planetary Science and Exploration",
    "Journal of Celestial Observations",
    "Intergalactic Studies",
    "Journal of Modern Astronomy",
    "Cosmology Today",
    "Advances in Planetary Science",
    "Astrogeology Journal",
    "Journal of Space and Time",
    "Journal of Cosmic Chemistry",
    "Exoplanet Research Journal",
    "Journal of Stellar Evolution",
    "Space Exploration and Technology",
    "Cosmic Horizons Review",
    "Journal of Observational Astronomy",
    "Astrophysical Journal Letters",
    "Journal of Interstellar Studies",
    "Galactic Astronomy",
    "Journal of Celestial Research",
    "Astroinformatics Journal",
    "Journal of Space Engineering",
    "Astrophysics Reports",
    "Journal of Solar Research",
    "Journal of Cosmic Phenomena",
    "International Journal of Space Science",
    "Journal of Astronomical Research",
    "Advances in Cosmic Studies",
    "Journal of Planetary Atmospheres",
    "Space Science and Exploration",
    "Journal of Deep Space Research",
    "Cosmology and Gravitation",
    "Journal of Stellar Dynamics",
    "Journal of Planetary Systems",
    "Astrophysical Methods",
    "Journal of Interplanetary Research",
    "Journal of Space Physics",
    "Cosmic Evolution Review",
    "Journal of Extragalactic Research",
    "Interstellar Astronomy",
    "Journal of Cosmic Origins",
    "Space and Universe Journal",
    "Advances in Interstellar Studies",
    "Journal of Space Exploration and Research",
    "Astronomy Research Journal",
    "Journal of Cosmic Energy",
    "Journal of Planetary Climates",
    "Space Science Insights",
    "Journal of Astrophysical Phenomena",
    "Cosmic Exploration Journal",
    "Journal of Celestial Phenomena",
    "Journal of Universe Studies",
    "Space, Stars, and Galaxies",
    "Journal of Cosmic Investigations",
    "Astronomical Research and Reviews",
    "International Journal of Space Exploration",
    "Journal of Advanced Astronomy",
    "Space and Beyond Journal",
    "Journal of Galactic Research",
    "Cosmology and Space Science"
]



# Collect scientist IDs from your existing scientists_df
scientist_ids = scientists_df["scientist_id"].tolist()

# Helper function to generate a random date between two boundaries
def random_date(start, end):
    delta = end - start
    random_days = random.randrange(delta.days)
    return start + timedelta(days=random_days)

start_date = datetime(2000, 1, 1)
end_date = datetime(2025, 12, 31)

papers_data = []
for i in range(1, num_papers + 1):
    # Generate a random publication date
    pub_date = random_date(start_date, end_date).strftime("%Y-%m-%d")
    
    # Optionally, create a random title referencing an integer or meteorite class
    random_num = random.randint(1, 50000)
    title = f"New Insights into Meteorite #{random_num}"
    
    # Pick a random scientist_id
    s_id = random.choice(scientist_ids)
    
    # Build the paper record
    papers_data.append({
        "paper_id": i,
        "title": title,
        "publication_date": pub_date,
        "journal_name": random.choice(journals),
        "scientist_id": s_id
    })

# Convert to DataFrame
papers_df = pd.DataFrame(papers_data)

# Inspect
print("Research Papers DataFrame shape:", papers_df.shape)
print(papers_df.head(10))


# (Optional) save to CSV
papers_df.to_csv("research_papers.csv", index=False)


Research Papers DataFrame shape: (38114, 5)
   paper_id                               title publication_date  \
0         1  New Insights into Meteorite #27953       2025-12-19   
1         2  New Insights into Meteorite #44901       2022-11-30   
2         3  New Insights into Meteorite #49437       2012-04-12   
3         4  New Insights into Meteorite #41695       2018-08-18   
4         5  New Insights into Meteorite #38554       2020-09-26   
5         6  New Insights into Meteorite #31227       2023-08-26   
6         7  New Insights into Meteorite #13783       2008-03-08   
7         8  New Insights into Meteorite #28153       2025-05-13   
8         9  New Insights into Meteorite #23026       2023-11-15   
9        10  New Insights into Meteorite #34743       2008-09-27   

                                journal_name  scientist_id  
0                   Astroinformatics Journal           899  
1                 Astronomy Research Journal         18876  
2                      I

In [183]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 38114 entries, 0 to 30784
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   meteorite_name  38114 non-null  object 
 1   meteorite_id    38114 non-null  int64  
 2   nametype        38114 non-null  object 
 3   mass            38114 non-null  float64
 4   fall_type       38114 non-null  object 
 5   fall_year       38114 non-null  int64  
 6   GeoLocation     38114 non-null  object 
 7   class_id        38114 non-null  int64  
 8   location_id     38114 non-null  int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 2.9+ MB


In [184]:
import pandas as pd
import random
from datetime import datetime, timedelta

# We'll assume you already have:
# - meteorites_df (or df) with a "meteorite_id" column
# - scientists_df with a "scientist_id" column

# 1. Set number of expedition records equal to the number of rows in your main DataFrame (df)
num_expeditions = len(df)

# 2. Get lists of valid meteorite and scientist IDs
meteorite_ids = df["meteorite_id"].tolist()
scientist_ids = scientists_df["scientist_id"].tolist()

# 3. Helper function to generate random dates
def random_date(start, end):
    delta = end - start
    random_days = random.randrange(delta.days)
    return start + timedelta(days=random_days)

start_date = datetime(1900, 1, 1)
end_date = datetime(2025, 12, 31)

# 4. Expanded list of possible expedition notes
expedition_notes = [
    "Collected sample from desert",
    "Expedition in polar region",
    "Field collection in forest",
    "Urban expedition",
    "Underwater exploration",
    "Mountain peak sampling",
    "Volcanic crater sampling",
    "Deep cave exploration",
    "Glacial core drilling",
    "Tropical rainforest expedition",
    "Arid plateau mission",
    "Archaeological site investigation",
    "Remote island retrieval",
    "Meteor crater excavation",
    "High-altitude balloon collection",
    "Ice shelf reconnaissance",
    "Desert dune survey"
]

# 5. Generate the Expeditions data
expeditions_data = []
for i in range(1, num_expeditions + 1):
    m_id = random.choice(meteorite_ids)
    s_id = random.choice(scientist_ids)
    expedition_dt = random_date(start_date, end_date).strftime("%Y-%m-%d")
    note = random.choice(expedition_notes)
    
    expeditions_data.append({
        "expedition_id": i,
        "meteorite_id": m_id,
        "scientist_id": s_id,
        "expedition_date": expedition_dt,
        "notes": note
    })

# 6. Create the Expeditions DataFrame
expeditions_df = pd.DataFrame(expeditions_data)

# 7. Inspect the result
print("Expeditions DataFrame shape:", expeditions_df.shape)
print(expeditions_df.head(10))
print("\nExpeditions table created successfully with the same number of rows as the main DataFrame!")

# 8. (Optional) Save to CSV
expeditions_df.to_csv("expeditions.csv", index=False)


Expeditions DataFrame shape: (38114, 5)
   expedition_id  meteorite_id  scientist_id expedition_date  \
0              1         44719         23717      1959-03-21   
1              2           799         16419      2024-11-05   
2              3         20925          7113      1970-12-30   
3              4          1607         13502      2018-01-13   
4              5          7150         24220      1933-10-05   
5              6         46788         25410      1947-08-16   
6              7         11557         13217      1944-03-27   
7              8         18832         31664      2017-04-10   
8              9         21312         19291      1923-06-13   
9             10          8850         33494      2009-04-08   

                              notes  
0        Field collection in forest  
1             Deep cave exploration  
2                Desert dune survey  
3        Expedition in polar region  
4            Underwater exploration  
5             Deep cave exp

In [185]:
# Create a copy of the main DataFrame and name it metrodf
metrodf = df.copy()

# Display information about metrodf
print("Information on metrodf:")
metrodf.info()


Information on metrodf:
<class 'pandas.core.frame.DataFrame'>
Index: 38114 entries, 0 to 30784
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   meteorite_name  38114 non-null  object 
 1   meteorite_id    38114 non-null  int64  
 2   nametype        38114 non-null  object 
 3   mass            38114 non-null  float64
 4   fall_type       38114 non-null  object 
 5   fall_year       38114 non-null  int64  
 6   GeoLocation     38114 non-null  object 
 7   class_id        38114 non-null  int64  
 8   location_id     38114 non-null  int64  
dtypes: float64(1), int64(4), object(4)
memory usage: 2.9+ MB


In [186]:
# Build metrodf by selecting only the fields you need
metrodf = df[[
    "meteorite_id",
    "meteorite_name",
    "mass",
    "fall_type",
    "fall_year",
    "class_id",
    "location_id"
]].copy()

# Verify
print("Information on metrodf:")
metrodf.info()

# Save the cleaned metrodf
metrodf.to_csv("metrodf_cleaned.csv", index=False)
print("metrodf_cleaned.csv written successfully!")


Information on metrodf:
<class 'pandas.core.frame.DataFrame'>
Index: 38114 entries, 0 to 30784
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   meteorite_id    38114 non-null  int64  
 1   meteorite_name  38114 non-null  object 
 2   mass            38114 non-null  float64
 3   fall_type       38114 non-null  object 
 4   fall_year       38114 non-null  int64  
 5   class_id        38114 non-null  int64  
 6   location_id     38114 non-null  int64  
dtypes: float64(1), int64(4), object(2)
memory usage: 2.3+ MB
metrodf_cleaned.csv written successfully!
